In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,70764.81,-0.15,0.07,-0.32,0.06,0.04,0.03,-0.02,-0.01
3,-0.15,12485.76,0.68,-0.01,0.08,-0.01,0.01,0.00,0.00
4,0.07,0.68,3127.90,0.01,0.02,-0.01,0.00,-0.00,0.00
5,-0.32,-0.01,0.01,394.70,0.01,0.01,-0.00,-0.00,-0.00
6,0.06,0.08,0.02,0.01,126.94,0.00,-0.00,0.00,-0.00
7,0.04,-0.01,-0.01,0.01,0.00,44.73,0.00,-0.00,-0.00
8,0.03,0.01,0.00,-0.00,-0.00,0.00,13.59,-0.00,-0.00
9,-0.02,0.00,-0.00,-0.00,0.00,-0.00,-0.00,2.82,-0.00
10,-0.01,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.34


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,0.00000,-0.00006,0.00002,0.00002,0.00003,-0.00004,-0.00004
3,-0.00001,1.00000,0.00011,-0.00000,0.00006,-0.00001,0.00001,0.00001,0.00004
4,0.00000,0.00011,1.00000,0.00001,0.00004,-0.00002,0.00001,-0.00002,0.00004
5,-0.00006,-0.00000,0.00001,1.00000,0.00004,0.00006,-0.00005,-0.00006,-0.00004
6,0.00002,0.00006,0.00004,0.00004,1.00000,0.00002,-0.00004,0.00002,-0.00003
7,0.00002,-0.00001,-0.00002,0.00006,0.00002,1.00000,0.00003,-0.00001,-0.00003
8,0.00003,0.00001,0.00001,-0.00005,-0.00004,0.00003,1.00000,-0.00003,-0.00000
9,-0.00004,0.00001,-0.00002,-0.00006,0.00002,-0.00001,-0.00003,1.00000,-0.00005
10,-0.00004,0.00004,0.00004,-0.00004,-0.00003,-0.00003,-0.00000,-0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.0953067885524062

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.56703556e+08 1.70657990e+07 1.04734269e+06 1.63038072e+04
 1.61963836e+03 1.88044752e+02 1.53738517e+01 4.99172382e-01
 9.02561444e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999726,-0.022743,-0.005509,-0.000698,-0.000222,-0.000078,-0.000023,-0.000005,-7.003549e-07
1,0.022513,0.999007,-0.038166,-0.004486,-0.001435,-0.000508,-0.000154,-0.000032,-3.690754e-06
2,0.006351,0.037915,0.999002,-0.021538,-0.006804,-0.002399,-0.000727,-0.000152,-1.800271e-05
3,0.000910,0.005135,0.020750,0.996968,-0.071087,-0.022284,-0.006691,-0.001391,-1.702070e-04
4,0.000348,0.001969,0.007888,0.068138,0.992188,-0.100673,-0.026256,-0.005372,-6.576340e-04
5,0.000154,0.000878,0.003507,0.028020,0.095237,0.988415,-0.112842,-0.020672,-2.514892e-03
6,0.000063,0.000358,0.001431,0.011359,0.035555,0.106647,0.987424,-0.109853,-1.256013e-02
7,0.000020,0.000111,0.000444,0.003520,0.010928,0.031074,0.104367,0.986294,-1.234305e-01
8,0.000005,0.000027,0.000109,0.000868,0.002696,0.007650,0.025177,0.121240,9.922703e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00027411, 0.00099347, 0.00099768, 0.00303186, 0.00781189,
       0.01158476, 0.01257626, 0.01370648, 0.00772969])